In [ ]:
FILE_PREFIX = 'assistment-2009-2010-skill'

params = {
    'max_iter': 50, # 'number of iterations'
    'show': True, # 'print progress')
    'init_std': 0.1, # 'weight initialization std'
    'init_lr': 0.01, # 'initial learning rate'
    'lr_decay': 0.75, # 'learning rate decay'
    'final_lr': 1E-5, # 'learning rate will not decrease after hitting this threshold'
    'momentum': 0.9, # 'momentum rate'
    'maxgradnorm': 50.0, # 'maximum gradient norm'
    'final_fc_dim': 50, # 'hidden state dim for final fc layer'
    'key_embedding_dim': 50, # 'question embedding dimensions')
    'batch_size': 64, # 'the batch size')
    'value_embedding_dim': 200, # 'answer and question embedding dimensions')
    'memory_size': 20, # 'memory size')
    'n_question': 123, # 'the number of unique questions in the dataset')
    'seqlen': 200, # 'the allowed maximum length of a sequence')
    'data_dir': '../dkt', # 'data directory')
    'data_name': '', # 'data set name')
    'load': FILE_PREFIX + 'dkvmn.params', # 'model file to load')
    'save': FILE_PREFIX + 'dkvmn.params' # 'path to save model')
}

params['lr'] = params['init_lr']
params['key_memory_state_dim'] = params['key_embedding_dim']
params['value_memory_state_dim'] = params['value_embedding_dim']

In [6]:
from load_data import Data

dat = Data(n_question=params['n_question'], seqlen=params['seqlen'], separate_char=',') 

train_data_path = params['data_dir'] + "/" + params['data_name'] + FILE_PREFIX + "train.txt"
test_data_path = params['data_dir'] + "/" + params['data_name'] + FILE_PREFIX + "test.txt"
train_data = dat.load_data(train_data_path)
test_data = dat.load_data(test_data_path)


In [ ]:
import optuna
from EduKTM import DKVMN

def objective(trial):
    params = {
        'max_iter': 10, # 'number of iterations'
        'show': False, # 'print progress')
        'init_std': trial.suggest_float('init_std', 0.01, 0.2), # 'weight initialization std'
        'init_lr': trial.suggest_float('init_lr', 0.001, 0.1), # 'initial learning rate'
        'lr_decay': trial.suggest_float('lr_decay', 0.5, 0.99), # 'learning rate decay'
        'final_lr': trial.suggest_float('final_lr', 1E-6, 1E-4), # 'learning rate will not decrease after hitting this threshold'
        'momentum': trial.suggest_float('momentum', 0.5, 0.99), # 'momentum rate'
        'maxgradnorm': trial.suggest_float('maxgradnorm', 10.0, 100.0), # 'maximum gradient norm'
        # 'final_fc_dim': 50, # 'hidden state dim for final fc layer'
        # 'key_embedding_dim': 50, # 'question embedding dimensions')
        'batch_size': 64, # 'the batch size')
        'value_embedding_dim': trial.suggest_int('value_embedding_dim', 50, 500), # 'answer and question embedding dimensions')
        'memory_size': trial.suggest_int('memory_size', 10, 100), # 'memory size')
        'n_question': 123, # 'the number of unique questions in the dataset')
        'seqlen': 200, # 'the allowed maximum length of a sequence')
        'data_dir': '../dkt', # 'data directory')
        'data_name': '', # 'data set name')
    }
    
    params['final_fc_dim'] = params['key_embedding_dim'] = trial.suggest_int('final_fc_dim', 10, 100) # 'hidden state dim for final fc layer'
    
    params['lr'] = params['init_lr']
    params['key_memory_state_dim'] = params['key_embedding_dim']
    params['value_memory_state_dim'] = params['value_embedding_dim']

    
    # Create and train the model
    model = DKVMN(n_question=params['n_question'],
                    batch_size=params['batch_size'],
                    key_embedding_dim=params['key_embedding_dim'],
                    value_embedding_dim=params['value_embedding_dim'],
                    memory_size=params['memory_size'],
                    key_memory_state_dim=params['key_memory_state_dim'],
                    value_memory_state_dim=params['value_memory_state_dim'],
                    final_fc_dim=params['final_fc_dim'])
    
    # Train and evaluate
    model.train(params, train_data)
    loss, auc, accuracy = model.eval(params, test_data)
    
    return auc  # Optimize for AUC


In [8]:
# Create and run the study
study = optuna.create_study(study_name=FILE_PREFIX+"dkvmn_importances", direction='maximize', storage="sqlite:///../studies.db", load_if_exists=True)
study.optimize(objective, n_trials=24, n_jobs=8)  # Adjust number of trials as needed

[I 2025-04-11 12:33:40,438] Using an existing study with name 'assistment-2009-2010-skilldkvmn_importances' instead of creating a new one.
Epoch 0:   0%|          | 0/54 [00:00<?, ?it/s]






































Epoch 0:   2%|▏         | 1/54 [00:03<03:15,  3.69s/it]




















Epoch 0:   4%|▎         | 2/54 [00:09<04:15,  4.92s/it]






















Epoch 0:   6%|▌         | 3/54 [00:14<04:05,  4.81s/it]






















Epoch 0:   7%|▋         | 4/54 [00:18<03:56,  4.73s/it]




















Epoch 0:   9%|▉         | 5/54 [00:22<03:28,  4.26s/it]



























Epoch 0:  11%|█         | 6/54 [00:25<03:16,  4.10s/it]




















Epoch 0:  13%|█▎        | 7/54 [00:28<02:46,  3.55s/it]



















Epoch 0:  15%|█▍        | 8/54 [00:31<02:42,  3.54s/it]

























Epoch 0:  17%|█▋        | 9/54 [00:34<02:23,  3.19s/it]























Epoch 0:  19%|█▊        | 10/54 [00:38<02:28,  3.38s/it]


















Epoch 1/10, loss : 0.60088, auc : 0.69631, accuracy : 0.69557














Epoch 0:  76%|███████▌  | 41/54 [02:13<00:38,  2.95s/it]































Epoch 0:  78%|███████▊  | 42/54 [02:15<00:33,  2.79s/it]



















Epoch 0:  80%|███████▉  | 43/54 [02:19<00:33,  3.03s/it]














Epoch 0: 100%|██████████| 54/54 [02:21<00:00,  2.62s/it]








Epoch 1/10, loss : 0.60524, auc : 0.67702, accuracy : 0.68610








Epoch 0:  81%|████████▏ | 44/54 [02:22<00:29,  2.99s/it]







Epoch 0: 100%|██████████| 54/54 [02:24<00:00,  2.68s/it]






...

Epoch 9/10, loss : 0.49159, auc : 0.80833, accuracy : 0.76221


Epoch 9: 100%|██████████| 54/54 [00:14<00:00,  3.68it/s]


Epoch 10/10, loss : 0.48891, auc : 0.81046, accuracy : 0.76326


Evaluating: 100%|██████████| 23/23 [00:03<00:00,  6.43it/s]
[I 2025-04-11 15:07:41,050] Trial 37 finished with value: 0.7577967933887637 and parameters: {'init_std': 0.13522735055727755, 'init_lr': 0.018798268726187745, 'lr_decay': 0.9180598599675551, 'final_lr': 3.643258162889654e-05, 'momentum': 0.6142514462970652, 'maxgradnorm': 83.4196484739815, 'value_embedding_dim': 225, 'memory_size': 43, 'final_fc_dim': 81}. Best is trial 35 with value: 0.7578473071133427.


valid auc : 0.79798, valid accuracy : 0.75780


In [9]:
# Print the results
print("Best trial:")
trial = study.best_trial
print(f"  Value (AUC): {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Value (AUC): 0.7578473071133427
  Params: 
    init_std: 0.1064392520682464
    init_lr: 0.020308044418767546
    lr_decay: 0.8769705774588257
    final_lr: 3.0584517647771867e-05
    momentum: 0.6193396295968937
    maxgradnorm: 84.68267098134736
    value_embedding_dim: 226
    memory_size: 65
    final_fc_dim: 76


In [10]:
# Visualize the importance of hyperparameters
param_importance = optuna.importance.get_param_importances(study)
print("\nParameter importance:")
for param, importance in param_importance.items():
    print(f"  {param}: {importance:.4f}")


Parameter importance:
  init_lr: 0.5235
  maxgradnorm: 0.2097
  momentum: 0.0836
  final_fc_dim: 0.0462
  init_std: 0.0359
  value_embedding_dim: 0.0315
  memory_size: 0.0275
  final_lr: 0.0241
  lr_decay: 0.0181


In [16]:
print(study.best_params)

{'init_std': 0.1064392520682464, 'init_lr': 0.020308044418767546, 'lr_decay': 0.8769705774588257, 'final_lr': 3.0584517647771867e-05, 'momentum': 0.6193396295968937, 'maxgradnorm': 84.68267098134736, 'value_embedding_dim': 226, 'memory_size': 65, 'final_fc_dim': 76}


In [ ]:
import optuna
from EduKTM import DKVMN

def refined_objective(trial):
    # Only optimize the most important parameters based on previous study
    refined_params = {
        'max_iter': 10,
        'show': False, # 'print progress')
        'init_std': 0.1064392520682464, # 'weight initialization std'
        'lr_decay': 0.8769705774588257, # 'learning rate decay'
        'final_lr': 3.0584517647771867E-05, # 'learning rate will not decrease after hitting this threshold'
        'final_fc_dim': 76, # 'hidden state dim for final fc layer'
        'key_embedding_dim': 76, # 'question embedding dimensions')
        'batch_size': 64, # 'the batch size')
        'value_embedding_dim': 226, # 'answer and question embedding dimensions')
        'memory_size': 65, # 'memory size')
        'n_question': 123, # 'the number of unique questions in the dataset')
        'seqlen': 200, # 'the allowed maximum length of a sequence')
        'data_dir': '../dkt', # 'data directory')
        'data_name': '', # 'data set name')
    }
    
    # Only optimize the top 3 most important parameters
    refined_params['init_lr'] = trial.suggest_float('init_lr', 0.01, 0.05)  # Narrow range around best value
    refined_params['maxgradnorm'] = trial.suggest_float('maxgradnorm', 50.0, 100.0)  # Narrow range around best value
    refined_params['momentum'] = trial.suggest_float('momentum', 0.5, 0.8)  # Narrow range around best value
    
    refined_params['lr'] = refined_params['init_lr']
    refined_params['key_memory_state_dim'] = refined_params['key_embedding_dim']
    refined_params['value_memory_state_dim'] = refined_params['value_embedding_dim']
    
    # Create and train the model
    model = DKVMN(n_question=refined_params['n_question'],
                  batch_size=refined_params['batch_size'],
                  key_embedding_dim=refined_params['key_embedding_dim'],
                  value_embedding_dim=refined_params['value_embedding_dim'],
                  memory_size=refined_params['memory_size'],
                  key_memory_state_dim=refined_params['key_memory_state_dim'],
                  value_memory_state_dim=refined_params['value_memory_state_dim'],
                  final_fc_dim=refined_params['final_fc_dim'])
    
    # Train and evaluate
    model.train(refined_params, train_data)
    loss, auc, accuracy = model.eval(refined_params, test_data)
    
    return auc  # Optimize for AUC

# Create and run the refined study
refined_study = optuna.create_study(study_name=FILE_PREFIX+"dkvmn_refined", 
                                   direction='maximize', 
                                   storage="sqlite:///../studies.db", 
                                   load_if_exists=True)
refined_study.optimize(refined_objective, n_trials=12, n_jobs=4)

# Print results
print("Best refined trial:")
refined_trial = refined_study.best_trial
print(f"  Value (AUC): {refined_trial.value}")
print("  Params: ")
for key, value in refined_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-04-12 10:12:48,595] Using an existing study with name 'assistment-2009-2010-skilldkvmn_refined' instead of creating a new one.
Epoch 0:   0%|          | 0/54 [00:00<?, ?it/s]




Epoch 0:   2%|▏         | 1/54 [00:04<03:35,  4.07s/it]



Epoch 0:   6%|▌         | 3/54 [00:09<02:43,  3.20s/it]


Epoch 0:   7%|▋         | 4/54 [00:12<02:25,  2.91s/it]


Epoch 0:   9%|▉         | 5/54 [00:15<02:22,  2.91s/it]


Epoch 0:  11%|█         | 6/54 [00:18<02:17,  2.86s/it]


Epoch 0:  13%|█▎        | 7/54 [00:21<02:16,  2.90s/it]


Epoch 0:  15%|█▍        | 8/54 [00:23<02:09,  2.82s/it]


Epoch 0:  17%|█▋        | 9/54 [00:26<02:07,  2.84s/it]


Epoch 0:  19%|█▊        | 10/54 [00:28<01:59,  2.71s/it]


Epoch 0:  20%|██        | 11/54 [00:31<01:55,  2.68s/it]


Epoch 0:  22%|██▏       | 12/54 [00:34<01:53,  2.69s/it]


Epoch 0:  24%|██▍       | 13/54 [00:37<01:51,  2.72s/it]


Epoch 0:  26%|██▌       | 14/54 [00:39<01:45,  2.64s/it]


Epoch 0:  28%|██▊       | 15/54 [00:42<01:47,  2.76s/

Epoch 1/50, loss : 0.59927, auc : 0.68229, accuracy : 0.68490



Epoch 0: 100%|██████████| 54/54 [02:07<00:00,  2.35s/it]


Epoch 1/50, loss : 0.59385, auc : 0.68753, accuracy : 0.68683


Epoch 1:   0%|          | 0/54 [00:00<?, ?it/s]

Epoch 0: 100%|██████████| 54/54 [02:07<00:00,  2.36s/it]


...




Epoch 35: 100%|██████████| 54/54 [02:34<00:00,  2.85s/it]


Epoch 36/50, loss : 0.48159, auc : 0.81492, accuracy : 0.76560


Epoch 36:   0%|          | 0/54 [00:00<?, ?it/s]


Epoch 35: 100%|██████████| 54/54 [02:36<00:00,  2.90s/it]


Epoch 36/50, loss : 0.51854, auc : 0.77957, accuracy : 0.74472


Epoch 36:   2%|▏         | 1/54 [00:03<03:26,  3.90s/it]


Epoch 36:   4%|▎         | 2/54 [00:07<03:23,  3.91s/it]


Epoch 36:   6%|▌         | 3/54 [00:11<03:16,  3.86s/it]


Epoch 36:   7%|▋         | 4/54 [00:17<03:46,  4.53s/it]


Epoch 36:   9%|▉         | 5/54 [00:20<03:23,  4.15s/it]


Epoch 36:  11%|█         | 6/54 [00:24<03:10,  3.96s/it]


Epoch 36:  13%|█▎        | 7/54 [00:30<03:37,  4.62s/it]




In [ ]:
params = {
    'max_iter': 50,
    'show': True, # 'print progress')
    'init_std': trial.params['init_std'], # 'weight initialization std'
    'init_lr': trial.params['init_lr'], # 'initial learning rate'
    'lr_decay': trial.params['lr_decay'], # 'learning rate decay'
    'final_lr': trial.params['final_lr'], # 'learning rate will not decrease after hitting this threshold'
    'momentum': trial.params['momentum'], # 'momentum rate'
    'maxgradnorm': trial.params['maxgradnorm'], # 'maximum gradient norm'
    'final_fc_dim': trial.params['final_fc_dim'], # 'hidden state dim for final fc layer'
    'key_embedding_dim': trial.params['final_fc_dim'], # 'question embedding dimensions')
    'batch_size': 64, # 'the batch size')
    'value_embedding_dim': trial.params['value_embedding_dim'], # 'answer and question embedding dimensions')
    'memory_size': trial.params['memory_size'], # 'memory size')
    'n_question': 123, # 'the number of unique questions in the dataset')
    'seqlen': 200, # 'the allowed maximum length of a sequence')
    'data_dir': '../dkt', # 'data directory')
    'data_name': '', # 'data set name')
    'load': FILE_PREFIX + 'dkvmn.params', # 'model file to load')
    'save': FILE_PREFIX + 'dkvmn.params' # 'path to save model')
}

params['lr'] = params['init_lr']
params['key_memory_state_dim'] = params['key_embedding_dim']
params['value_memory_state_dim'] = params['value_embedding_dim']

In [14]:
from EduKTM import DKVMN

dkvmn = DKVMN(n_question=params['n_question'],
                  batch_size=params['batch_size'],
                  key_embedding_dim=params['key_embedding_dim'],
                  value_embedding_dim=params['value_embedding_dim'],
                  memory_size=params['memory_size'],
                  key_memory_state_dim=params['key_memory_state_dim'],
                  value_memory_state_dim=params['value_memory_state_dim'],
                  final_fc_dim=params['final_fc_dim'])

dkvmn.train(params, train_data)
dkvmn.save(params['save'])

Epoch 0:   0%|          | 0/54 [00:00<?, ?it/s]

Epoch 0: 100%|██████████| 54/54 [00:24<00:00,  2.24it/s]


Epoch 1/100, loss : 0.60061, auc : 0.67668, accuracy : 0.68086


Epoch 1: 100%|██████████| 54/54 [00:22<00:00,  2.37it/s]


Epoch 2/100, loss : 0.55598, auc : 0.74128, accuracy : 0.71908


...

Epoch 98/100, loss : 0.45532, auc : 0.83628, accuracy : 0.77851


Epoch 98: 100%|██████████| 54/54 [00:20<00:00,  2.63it/s]


Epoch 99/100, loss : 0.45520, auc : 0.83631, accuracy : 0.77844


Epoch 99: 100%|██████████| 54/54 [00:20<00:00,  2.66it/s]

Epoch 100/100, loss : 0.45552, auc : 0.83621, accuracy : 0.77823


In [15]:
dkvmn.load(params['load'])
dkvmn.eval(params, test_data)

Evaluating: 100%|██████████| 23/23 [00:34<00:00,  1.48s/it]

valid auc : 0.78771, valid accuracy : 0.74789


(0.5128569512263589, 0.7478860006263702, np.float64(0.7877146741009113))